## Intro

This notebook provides a quickstart guide to the pylume python library.

In [ ]:
!pip uninstall pylume -y

In [ ]:
!pip install pylume

In [ ]:
!pip install pydantic

In [ ]:
# If locally installed, use this instead:
# !poetry install
# !poetry build
!pip uninstall pylume -y && pip install ./dist/pylume-0.1.0-py3-none-any.whl --force-reinstall

In [3]:
import asyncio
from pylume import (
    PyLume, 
    ImageRef, 
    VMRunOpts, 
    SharedDirectory, 
    VMConfig,
    VMUpdateOpts
)

### Get latest IPSW URL from Apple Server

This is used to create a new macOS VM by providing the downloaded IPSW file path to the `ipsw` argument in the `create_vm` method.

In [ ]:
async def get_ipsw():
    async with PyLume(port=7777) as pylume:
        url = await pylume.get_latest_ipsw_url()
        print(f"Latest IPSW URL: {url}")

await get_ipsw()

### Create a new VM

#### macOS

An IPSW file path is required to create a new macOS VM. To fetch automatically the latest IPSW during the VM creation, use `ipsw="latest"`.

In [ ]:
async def create_macos_vm():
    async with PyLume() as pylume:
        vm_config = VMConfig(
            name="macos-vm",
            os="macOS",
            cpu=4,
            memory="4GB",
            disk_size="40GB",
            display="1024x768",
            ipsw="latest"
        )
        await pylume.create_vm(vm_config)

await create_macos_vm()

#### Linux

To create a new Linux VM, use the `os="linux"` argument in the `VMConfig` class. Note that this doesn't set up any Linux distribution, it just creates a VM with a Linux kernel.

In [ ]:
async def create_linux_vm():
    async with PyLume() as pylume:
        vm_config = VMConfig(
            name="linux-vm",
            os="linux",
            cpu=2,
            memory="4GB",
            disk_size="25GB",
            display="1024x768"
        )
        await pylume.create_vm(vm_config)

await create_linux_vm()

### Pull an image from ghcr.io/trycua

Cua provides pre-built images for macOS and Linux.

In [ ]:
async def pull_macos_image():
    async with PyLume() as pylume:
        image_ref = ImageRef(
            image="macos-sequoia-vanilla",
            tag="15.2",
            registry="ghcr.io",
            organization="trycua"
        )
        await pylume.pull_image(image_ref, name="macos-sequoia-vanilla")

await pull_macos_image()

### Run

Run a VM by providing the `VMRunConfig` object.

In [ ]:
async def run_vm():
    async with PyLume() as pylume:
        vm_name = "macos-sequoia-vanilla"
        run_opts = VMRunOpts(
            no_display=False,
            shared_directories=[
                SharedDirectory(
                    host_path="/Users/<USER>/Shared",
                    read_only=False
                )
            ]
        )
        await pylume.run_vm(vm_name, run_opts)

await run_vm()

### List existing VMs

VMs are stored in the `~/.lume` directory.

In [ ]:
async with PyLume() as pylume:
    vms = await pylume.list_vms()

### Get VM status

In [ ]:
async with PyLume() as pylume:
    status = await pylume.get_vm("macos-sequoia-vanilla")
    print(status)

### Update VM Settings

In [ ]:
update_config = VMUpdateOpts(
    cpu=8,
    memory="8GB"
)
async with PyLume() as pylume:
    await pylume.update_vm("macos-sequoia-vanilla", update_config)

### Stop a VM

In [ ]:
async with PyLume() as pylume:
    await pylume.stop_vm("macos-sequoia-vanilla")

### Delete a VM

In [ ]:
async with PyLume() as pylume:
    await pylume.delete_vm("linux-vm")

### Images

List the images available locally

In [ ]:
async with PyLume() as pylume:
    await pylume.get_images()